# Classification Experiment
## Import libraries

In [ ]:
import os
import random
import datetime
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset
import yaml
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import datetime as dt
from tqdm import tqdm

In [ ]:
def create_save_dir(args):
    if False: #"subject_num" in args:
        if type(args['subject_num']) is not list:
            args['subject_num'] = [args['subject_num']]

        subjs_str = ','.join(str(x) for x in args['subject_num'])
        args['save_dir'] = os.path.join(args['save_dir'], subjs_str, datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S'))
    else:
        args['save_dir'] = os.path.join(args['save_dir'], datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S'))
    if not os.path.isdir(args['save_dir']):
        os.makedirs(args['save_dir'])

In [ ]:
class Metrics:
    def __init__(self, column_names):
        column_names.insert(0, "time_stamp")
        self.df = pd.DataFrame(columns=column_names)

    def add_row(self, row_list):
        row_list.insert(0, str(dt.datetime.now()))
        # print(row_list)
        self.df.loc[len(self.df)] = row_list

    def save_to_csv(self, filepath):
        self.df.to_csv(filepath, index=False)

## Load raw dataset

In [ ]:
class HIDataset(Dataset):
    def __init__(self, X, Y):
        self.data, self.targets = X, Y
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        x = self.data[idx]  # (16, 240) → (canales, serie de tiempo)
        y = self.targets[idx]  # Clase correspondiente
        # Convierte NumPy a tensores de PyTorch
        x = torch.tensor(x, dtype=torch.float32)
        y = torch.tensor(y, dtype=torch.long)
        return x, y

In [ ]:
def sel_exp(caso, data, labels):
    objeto = {
        'Multi': {'aceptar': 0, 'cancelar': 1, 'arriba': 2, 'abajo': 3, 'derecha': 4, 'izquierda': 5,
                  'hola': 6, 'ayuda': 7, 'gracias': 8, 'a': 9, 'e': 10, 'i': 11, 'o': 12, 'u': 13
                  },
        'PalabraVsVocal': {'aceptar': '', 'cancelar': '', 'arriba': '', 'abajo': '', 'derecha': '', 'izquierda': 0,
                           'hola': '', 'ayuda': '', 'gracias': '', 'a': '', 'e': '', 'i': 1, 'o': '', 'u': ''
                           },
        'PalabrasVsVocal': {'aceptar': '', 'cancelar': '', 'arriba': 0, 'abajo': 0, 'derecha': 0, 'izquierda': 0,
                            'hola': 0, 'ayuda': '', 'gracias': '', 'a': 1, 'e': 1, 'i': 1, 'o': 1, 'u': 1
                            },
        'Palabras': {'aceptar': 0, 'cancelar': 1, 'arriba': 2, 'abajo': 3, 'derecha': 4, 'izquierda': 5,
                     'hola': 6, 'ayuda': 7, 'gracias': 8, 'a': '', 'e': '', 'i': '', 'o': '', 'u': ''
                     },
        'PalabrasCoretto': {'aceptar': 0, 'cancelar': 1, 'arriba': 2, 'abajo': 3, 'derecha': 4, 'izquierda': 5,
                     'hola': '', 'ayuda': '', 'gracias': '', 'a': '', 'e': '', 'i': '', 'o': '', 'u': ''
                     },
        'Vocales': {'aceptar': '', 'cancelar': '', 'arriba': '', 'abajo': '', 'derecha': '', 'izquierda': '',
                    'hola': '', 'ayuda': '', 'gracias': '', 'a': 0, 'e': 1, 'i': 2, 'o': 3, 'u': 4
                    },
        'VocalesASU': {'aceptar': '', 'cancelar': '', 'arriba': '', 'abajo': '', 'derecha': '', 'izquierda': '',
                    'hola': '', 'ayuda': '', 'gracias': '', 'a': 0, 'e': '', 'i': 1, 'o': '', 'u': 2
                    },
        'CortaVsLarga': {'aceptar': '', 'cancelar': '', 'arriba': '', 'abajo': '', 'derecha': '', 'izquierda': 0,
                           'hola': 1, 'ayuda': '', 'gracias': '', 'a': '', 'e': '', 'i': '', 'o': '', 'u': ''
                           },
        'PalabrasTOL': {'aceptar': '', 'cancelar': '', 'arriba': 0, 'abajo': 1, 'derecha': 2, 'izquierda': 3,
                     'hola': '', 'ayuda': '', 'gracias': '', 'a': '', 'e': '', 'i': '', 'o': '', 'u': ''
                     },

    }
    if caso=='Multi':
        X=np.vstack(data["hi"])
        Y = np.repeat(np.arange(14), data["hi"].shape[1])  # (336,)
    elif caso=='HIvsHA':
        X1=np.vstack(data["hi"][:,:data["ha"].shape[1],:,:])
        X2=np.vstack(data["ha"])
        X=np.concatenate((X1, X2))
        Y = np.repeat(np.arange(2), X2.shape[0])
    else:
        datos=[]
        etiquetas=[]
        for i in range(0,14):
            if (objeto[caso][labels[i]]!=''):
                datos.append(data["hi"][i,:,:,:])
                for j in range(0,data["hi"].shape[1]):
                    etiquetas.append(objeto[caso][labels[i]])
        X=np.vstack(datos)
        Y = etiquetas  # (336,)

    X_train, X_test, Y_train, Y_test =train_test_split(X, Y, test_size=0.2, random_state=42, stratify=Y)
    return X_train, X_test, Y_train, Y_test

In [ ]:
def load_eeg_data(args, batch_size=32):
    # Cargar el archivo NumPy
    data = np.load(f's-{args['subject_num']}-prepros-ASU.npz')
    labels = np.load(f's-{args['subject_num']}-prepros-ASU.npz')['labels']
    # División de entrenamiento y prueba
    X_train, X_test, Y_train, Y_test=sel_exp(args["caso"], data, labels)
    # Crear DataLoaders
    train_dataset = HIDataset(X_train, Y_train)
    test_dataset = HIDataset(X_test, Y_test)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, test_loader


## The model

Deep neural network based on a basic Transformer.

In [ ]:
class NetTraST(nn.Module):
    def __init__(self, args):
        super(NetTraST, self).__init__()
        self.batch_norm1 = nn.BatchNorm1d(args['vocab_size'])
        p = args['kernel_size'] // 2
        self.conv1 = nn.Conv1d(in_channels=args['vocab_size'], out_channels=args['kernel_num'], kernel_size=args['kernel_size'], stride=1, padding=p)

        self.conv2 = nn.Conv1d(in_channels=args['embed_dim'], out_channels=args['kernel_num'], kernel_size=args['kernel_size'], stride=1, padding=p)
        self.upsamp = nn.Upsample((args['embed_dim']))

        self.rrelu = nn.RReLU(0.1, 0.3)
        nl=3 #args['num_layers']//2
        self.spatial_tra = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(
                d_model=args['embed_dim'],
                nhead=args['nhead'],
                dim_feedforward=args['dim_feedforward'],
            ),
            num_layers=nl,
        )
        self.temporal_tra = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(
                d_model=args['vocab_size'],
                nhead=args['nhead'],
                dim_feedforward=args['dim_feedforward'],
            ),
            num_layers=nl,
        )
        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(
                d_model=args['kernel_num'],
                nhead=args['nhead'],
                dim_feedforward=args['dim_feedforward'],
            ),
            num_layers=args['num_layers'],
        )
        self.batch_norm3 = nn.BatchNorm1d(args['kernel_num'])
        self.fl = nn.Flatten()
        self.fc1 = nn.Linear(args['kernel_num']*args['embed_dim'], args['kernel_num'])
        self.dropout = nn.Dropout(args['dropout'])
        self.fc2 = nn.Linear(args['kernel_num'], args['class_num'])

    def forward(self, x):
        #print(f'input {x.shape}')
        x = self.batch_norm1(x)
        #print(f'batch1 {x.shape}')
        x1 = self.conv1(x)
        #print(f'convSpa {x1.shape}')
        x1 = self.spatial_tra(x1)
        #print(f'transSpatial {x1.shape}')
        x2 = x.permute(0, 2, 1)
        #print(f'permute tempo {x2.shape}')
        x2 = self.conv2(x2)
        #print(f'convTemp {x2.shape}')
        x2 = self.temporal_tra(x2)
        #print(f'transTemp {x2.shape}')
        x2 = self.upsamp(x2)
        #print(f'upsamp {x2.shape}')

        x = x1+x2
        #print(f'concat {x.shape}')
        x = x.permute(2, 0, 1)  # Change the shape to (sequence_length, batch_size, input_size)
        #print(f'permute concat {x.shape}')
        x = self.transformer(x)
        #print(f'trans {x.shape}')
        x = x.permute(1, 2, 0)  # Change the shape to (batch_size, input_size, sequence_length)
        #print(f'permute trans {x.shape}')
        x = self.batch_norm3(x)
        #print(f'batch2 {x.shape}')
        x = self.fl(x)
        #print(f'flatten {x.shape}')
        x = self.rrelu(self.fc1(x))
        #print(f'relu {x.shape}')
        x = self.dropout(x)
        #print(f'dropout {x.shape}')
        x = self.fc2(x)
        #print(f'fc2 {x.shape}')
        return x

## Training and Evaluation functions

In [ ]:
def evaluation_raw(args, model, test_loader, criterion):
    # Evaluation
    model.eval()
    all_labels = []
    all_predictions = []
    with torch.no_grad():
        tot_loss = 0
        test_corrects = torch.tensor(0, device=args['device'])
        for inputs, labels in test_loader:
            inputs = inputs.to(args['device'])
            labels = labels.to(args['device'])
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            _, predicted = torch.max(outputs, 1)
            corrects = (torch.max(outputs, 1)[1].view(labels.size()).data == labels.data).sum()
            test_corrects += corrects
            tot_loss += loss

            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())

        ts_acc = 100.0 * test_corrects/len(test_loader.dataset)
        #print(f'Test Accuracy: {ts_acc:.4f}, Test loss: {tot_loss:.6f}')
    return ts_acc.cpu().item(), tot_loss,  all_labels, all_predictions


def train_raw(args, model, train_loader, optimizer, criterion, test_loader, metrics, subj, scheduler=None):
    #metrics = Metrics(["epoch", "lr", "train_loss", "train_acc", "test_loss", "test_acc", "best_test_acc"])
    # Training loop
    best_acc = 0.0
    best_CM=[]
    patience_counter = 0
    steps = 0
    loop_obj = tqdm(range(args['epochs']))
    loop_obj.set_postfix_str(f"Best val. acc.: {best_acc:.4f}, pat={patience_counter}") # Adds text after progressbar
    for epoch in loop_obj:
        loop_obj.set_description(f"Subj.: {subj}, Training epoch: {epoch+1}")  # Adds text before progessbar
        train_corrects = torch.tensor(0, device=args['device'])
        tot_loss = 0
        model.train()
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            inputs = inputs.to(args['device'])

            labels = labels.to(args['device'])
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            corrects = (torch.max(outputs, 1)[1].view(labels.size()).data == labels.data).sum()
            train_corrects += corrects
            tot_loss += loss
            loss.backward()
            optimizer.step()

        if scheduler: scheduler.step()

        tr_acc = 100.0 * train_corrects/len(train_loader.dataset)
        # Validation
        dev_acc, test_loss, all_labels, all_predictions = evaluation_raw(args, model, test_loader, criterion)

        if dev_acc > best_acc:
            best_acc = dev_acc
            best_CM=[all_labels, all_predictions]
            patience_counter = 0
            #torch.save(model, os.path.join(args['save_dir'], f"{model.__class__.__name__}_model_best.pt"))
            loop_obj.set_postfix_str(f"Best val. acc.: {best_acc:.4f}, pat={patience_counter}")
        else:
            patience_counter += 1
            loop_obj.set_postfix_str(f"Best val. acc.: {best_acc:.4f}, pat={patience_counter}")


        #EP=args['epochs']
        #print(f'Epoch [{epoch+1}/{EP}] Tr. Loss: {tot_loss.item():.4f} Val. Accuracy: {dev_acc:.4f} Best Val. Accuracy: {best_acc:.4f}')
        lr=optimizer.param_groups[0]["lr"]
        metrics.add_row([epoch+1, lr, tot_loss.cpu().item(), tr_acc.cpu().item(), test_loss.cpu().item(), dev_acc, best_acc])
        metrics.save_to_csv(os.path.join(args['save_dir'], "metrics_classifciation.csv"))

        if patience_counter > args['early_stopping_patience']:

            break
    return best_acc, best_CM

## Run an experiment

- change the parameter '*subject_num*' in the *args* dictionary to change the subject to one of the following
  - 'subject_num': [1]
  - 'subject_num': [2]
  - ...
  - 'subject_num': [10]  

In [ ]:
def get_default_args():
    args = {
        'class_num': 3,
        'dropout': 0.1 ,
        'nhead': 4 ,
        'dim_feedforward': 125 ,
        'num_layers': 5 ,
        'embed_dim': 240,
        'vocab_size': 16,
        'kernel_num': 128,
        'kernel_size': 3,
        'batch_size': 62 ,
        'epochs': 1000 ,
        'early_stopping_patience': 300 ,
        'lr': 0.001 ,
        'device': 'cuda:0' if torch.cuda.is_available() else 'cpu',
        'save_dir': 'experiments/',
        'subject_num': 0, # [1,2,3,4,5,6,7,8,9,10]
        'caso':'Multi'
    }

    return args

In [ ]:
def single_run_onevsones(caso):
    args = get_default_args()
    create_save_dir(args)
    labels=['aceptar' ,'cancelar' ,'arriba', 'abajo', 'derecha' ,'izquierda', 'hola','ayuda', 'gracias' ,'a' ,'e', 'i' ,'o' ,'u']
    # For all the results
    metrics = Metrics(["epoch", "lr", "train_loss", "train_acc", "test_loss", "test_acc", "best_test_acc"])
    resres=[]
    resresCM=[]
    args['caso']=caso
    for sub in range(1,17):
        results = {}
        results_CM = {}
        for trial in range(0,10):
            args['subject_num'] = sub
            model = NetTraST(args)
            model = model.to(args['device'])

            # Define the loss function and optimizer
            criterion = nn.CrossEntropyLoss()
            optimizer = optim.Adam(model.parameters()) #, lr=args['lr'])

            train_loader, test_loader = load_eeg_data(args)

            best_acc, best_CM = train_raw(args, model, train_loader, optimizer, criterion, test_loader, metrics, sub)
            #torch.save(model, os.path.join(args['save_dir'], f"{model.__class__.__name__}_model_last.pt"))
            results[trial] = best_acc
            results_CM[trial] = best_CM
            accs = np.array(list(results.values()))
            print(f'acc: mean={np.mean(accs):.4f}%, std={np.std(accs):.4f}%')

        # Print subject results
        str = f"RESULTS FOR 10 TRIALS {labels[5]} vs {labels[11]}\n"
        str += '--------------------------------\n'
        for key, value in results.items():
            str += f'Prueba {key}: {value:.4f} %\n'
        accs = np.array(list(results.values()))
        str += f'mean: {np.mean(accs):.4f}%, std: {np.std(accs):.4f}%\n'
        print(str)
        resres.append(results)
        resresCM.append(results_CM)
    return resres,resresCM

In [ ]:
caso='VocalesASU' # Multi, PalabraVsVocal , PalabrasVsVocal , Palabras , Vocales
resres,resresCM = single_run_onevsones(caso)

/home/graciela/anaconda3/envs/LRSG/lib/python3.12/site-packages/torch/nn/modules/transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(
Subj.: 1, Training epoch: 306:  30%|███       | 305/1000 [00:43<01:39,  6.98it/s, Best val. acc.: 66.6667, pat=301]
/home/graciela/anaconda3/envs/LRSG/lib/python3.12/site-packages/torch/nn/modules/transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


acc: mean=66.6667%, std=0.0000%


Subj.: 1, Training epoch: 307:  31%|███       | 306/1000 [00:44<01:41,  6.86it/s, Best val. acc.: 66.6667, pat=301]
/home/graciela/anaconda3/envs/LRSG/lib/python3.12/site-packages/torch/nn/modules/transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


acc: mean=66.6667%, std=0.0000%


Subj.: 1, Training epoch: 307:  31%|███       | 306/1000 [00:45<01:43,  6.71it/s, Best val. acc.: 53.3333, pat=301]
/home/graciela/anaconda3/envs/LRSG/lib/python3.12/site-packages/torch/nn/modules/transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


...

acc: mean=45.8333%, std=3.9965%


Subj.: 16, Training epoch: 311:  31%|███       | 310/1000 [02:24<05:20,  2.15it/s, Best val. acc.: 40.0000, pat=301]
/home/graciela/anaconda3/envs/LRSG/lib/python3.12/site-packages/torch/nn/modules/transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


acc: mean=45.1852%, std=4.1903%


Subj.: 16, Training epoch: 306:  30%|███       | 305/1000 [02:22<05:24,  2.14it/s, Best val. acc.: 53.3333, pat=301]

acc: mean=46.0000%, std=4.6667%
RESULTS FOR 10 TRIALS izquierda vs i
--------------------------------
Prueba 0: 46.6667 %
Prueba 1: 53.3333 %
Prueba 2: 40.0000 %
Prueba 3: 46.6667 %
Prueba 4: 40.0000 %
Prueba 5: 46.6667 %
Prueba 6: 46.6667 %
Prueba 7: 46.6667 %
Prueba 8: 40.0000 %
Prueba 9: 53.3333 %
mean: 46.0000%, std: 4.6667%



In [ ]:
print(resresCM)

[{0: [[1, 0, 1, 1, 2, 1, 1, 0, 2, 2, 2, 0, 2, 0, 0], [1, 2, 1, 1, 2, 1, 1, 2, 2, 1, 1, 0, 2, 1, 0]], 1: [[1, 0, 1, 1, 2, 1, 1, 0, 2, 2, 2, 0, 2, 0, 0], [1, 1, 1, 1, 1, 1, 1, 0, 2, 1, 1, 0, 2, 1, 0]], 2: [[1, 0, 1, 1, 2, 1, 1, 0, 2, 2, 2, 0, 2, 0, 0], [2, 0, 1, 2, 0, 1, 2, 0, 2, 0, 2, 2, 2, 2, 0]], 3: [[1, 0, 1, 1, 2, 1, 1, 0, 2, 2, 2, 0, 2, 0, 0], [1, 0, 1, 2, 0, 1, 2, 0, 2, 2, 1, 1, 2, 1, 0]], 4: [[1, 0, 1, 1, 2, 1, 1, 0, 2, 2, 2, 0, 2, 0, 0], [0, 2, 0, 1, 2, 0, 2, 0, 2, 2, 1, 0, 2, 0, 2]], 5: [[1, 0, 1, 1, 2, 1, 1, 0, 2, 2, 2, 0, 2, 0, 0], [0, 2, 1, 1, 0, 1, 2, 0, 2, 0, 0, 0, 2, 0, 0]], 6: [[1, 0, 1, 1, 2, 1, 1, 0, 2, 2, 2, 0, 2, 0, 0], [1, 0, 1, 2, 2, 0, 2, 0, 2, 2, 1, 1, 2, 2, 0]], 7: [[1, 0, 1, 1, 2, 1, 1, 0, 2, 2, 2, 0, 2, 0, 0], [1, 0, 1, 2, 0, 1, 2, 2, 2, 2, 1, 1, 0, 0, 0]], 8: [[1, 0, 1, 1, 2, 1, 1, 0, 2, 2, 2, 0, 2, 0, 0], [0, 0, 1, 1, 0, 1, 2, 0, 2, 0, 1, 0, 2, 0, 0]], 9: [[1, 0, 1, 1, 2, 1, 1, 0, 2, 2, 2, 0, 2, 0, 0], [1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 2, 0, 0]]}, {0: [[1

In [ ]:
import json

# Lista de diccionarios
data = resres
def convertir_a_json(obj):
    if isinstance(obj, np.integer):
        return int(obj)
    elif isinstance(obj, np.floating):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    raise TypeError(f"Tipo {type(obj)} no es serializable en JSON")# Guardar en un archivo .txt en formato JSON
with open(f"resultados-{caso}.txt", "w", encoding="utf-8") as file:
    json.dump(data, file, ensure_ascii=False, indent=4, default=convertir_a_json)

# Lista de diccionarios
data = resresCM
# Guardar en un archivo .txt en formato JSON
with open(f"resultados-{caso}-cm.txt", "w", encoding="utf-8") as file:
    json.dump(data, file, ensure_ascii=False, indent=4, default=convertir_a_json)